## Objective: identify potential monthly mortgage expenses for each region based on factors which are primarily monthly family income and rented value of the real estate.
- predict the potential demand in dollars amount of loan for each of the region in the USA

In [ ]:
# required packages
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.disable_max_rows()
import seaborn as sns

In [ ]:
#pip install pandasql

# Data Science project stages
1. Business understanding
2. Data understanding
3. Data preparation
4. Modeling
5. Evaluation
6. Deployment

## 2. Data understanding
- Explore Data

In [ ]:
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')
test_df.shape, train_df.shape

## 3. Data preparation

### Analyze Dataset

#### Handling Row Duplication

In [ ]:
# check for duplicates on train set
print(train_df.duplicated().sum())
print(train_df.duplicated(subset=['state', 'city']).sum())

# check for duplicates on test set
print(test_df.duplicated().sum())
print(test_df.duplicated(subset=['state', 'city']).sum())

By looking only at these two columns instead of the entire column set, we can see that the number of duplicate rows has increased. This means that there are rows that have the exact same values as these two columns but have different values in other columns, which means they may be different records. It is better to use all columns to identity duplicate records.

In [ ]:
# use UID as index

# train set
train_df.set_index(keys=['UID'], inplace=True)

# test set
test_df.set_index(keys=['UID'], inplace=True)

In [ ]:
# drop duplicates in order to avoid overfitting bias on model.

# test set
train_df = train_df.drop_duplicates(keep='first')
print(train_df.shape)

# test set
test_df = test_df.drop_duplicates(keep='first')
print(test_df.shape)

In [ ]:
# check for null values

# train set
print(train_df.isna().sum().sum())

# test set
print(test_df.isna().sum().sum())

#### Converting data types

In [ ]:
# train set 
train_obj_df = train_df.select_dtypes(include='object')

# test set
test_obj_df = test_df.select_dtypes(include='object')

In [ ]:
# list of unique values for each variable of the 'object' type

# train set
train_obj_cols = train_obj_df.columns
#for i in train_obj_cols:
    #print(i)
    #print(train_obj_df[i].unique())
    
# test set
test_obj_cols = train_obj_df.columns
#for i in test_obj_cols:
    #print(i)
    #print(test_obj_df[i].unique())

All these columns have a finite number of unique values that are composed of text, which shows that they are categorical variables.

In [ ]:
# train set: convert to object from numerical
train_vars_convert = ['COUNTYID', 'STATEID', 'zip_code', 'area_code']
for i in train_vars_convert:
    train_df[i] = train_df[i].astype('object')
print(train_df.select_dtypes(include='object').head())

# test set: convert to object from numerical
test_vars_convert = ['COUNTYID', 'STATEID', 'zip_code', 'area_code']
for i in test_vars_convert:
    test_df[i] = test_df[i].astype('object')
print(test_df.select_dtypes(include='object').head())

##### Features to drop
- Theses features only have 1 value so they do not add any variance to the data.

In [ ]:
# train set
train_features_to_drop = []
for column in train_df:
    if (train_df[column].nunique() == 1):
        train_features_to_drop.append(column)
print(train_features_to_drop)

# test set
test_features_to_drop = []
for column in test_df:
    if (test_df[column].nunique() == 1):
        test_features_to_drop.append(column)
print(test_features_to_drop)

In [ ]:
# dropping features with nunique() = 1 and BLOCKID; [has no values]

# train set
train_df.drop(columns=['SUMLEVEL', 'primary', 'BLOCKID'], inplace=True)
print(train_df.shape)

# test set
test_df.drop(columns=['SUMLEVEL', 'primary', 'BLOCKID'], inplace=True)
print(test_df.shape)

### Missing value treatment

In [ ]:
# check for train set missing values
print('Train missing values total: {}'.format(train_df.isna().sum().sum()))

# check for test set missing values
print('Test missing values total: {}'.format(test_df.isna().sum().sum()))

In [ ]:
# list of train set columns with missing values
train_cols_missing_vals = []
for i in train_df.columns:
    if (train_df[i].isna().sum() >= 1):
        train_cols_missing_vals.append(i)
        
# list of test set columns with missing values
test_cols_missing_vals = []
for i in test_df.columns:
    if (test_df[i].isna().sum() >= 1):
        test_cols_missing_vals.append(i)
        
print(len(train_cols_missing_vals), len(test_cols_missing_vals))

#### Missing value treatment for categorical columns

In [ ]:
# missing categorical values train set
train_missing_cat_vars = []
for i in train_cols_missing_vals:
    if ((train_df[i].dtype == 'object')):
        train_missing_cat_vars.append(i)


# missing categorical values test set
test_missing_cat_vars = []
for i in test_cols_missing_vals:
    if ((test_df[i].dtype == 'object')):
        test_missing_cat_vars.append(i)

print(len(train_missing_cat_vars), len(test_missing_cat_vars))

There aren't any missing values for categorical features. We will confirm by checking the df.

In [ ]:
print('Train & Test missing categorical values:{} \n'.format(train_df.select_dtypes(include=('object', 'category')).isna().sum()))
print(test_df.select_dtypes(include=('object', 'category')).isna().sum())

#### Missing value treatment for numerical columns.

In [ ]:
# train set missing numerical values 
train_missing_num_vars = []
for i in train_cols_missing_vals:
    if (train_df[i].dtype != 'object'):
        train_missing_num_vars.append(i)
#train_missing_num_vars

# test set missing numerical values 
test_missing_num_vars = []
for i in test_cols_missing_vals:
    if (test_df[i].dtype != 'object'):
        test_missing_num_vars.append(i)
#test_missing_num_vars

print('Train & Test missing values for numerical columns respectively: {} {}'.format(len(train_missing_num_vars), len(test_missing_num_vars)))

#### Distribution of columns

In [ ]:
# train_skewness_df
train_skewness_df = pd.DataFrame()
for i in train_missing_num_vars:
    train_skewness_df[i] = train_df[i]
    
# test_skewness_df
test_skewness_df = pd.DataFrame()
for i in test_missing_num_vars:
    test_skewness_df[i] = test_df[i]
#test_skewness_df.head()

train_skewness_df.head(2)

In [ ]:
# train set column list
train_num_cols_with_na = train_skewness_df.keys().tolist()
print(len(train_num_cols_with_na))

# test set column list
test_num_cols_with_na = test_skewness_df.keys().tolist()
print(len(test_num_cols_with_na))

In [ ]:
# train set value list
train_num_skeweness_values = []
for i in train_num_cols_with_na:
    train_num_skeweness_values.append(train_df[i].skew())
print(len(train_num_skeweness_values))

# test set value list
test_num_skeweness_values = []
for i in test_num_cols_with_na:
    test_num_skeweness_values.append(test_df[i].skew())
print(len(test_num_skeweness_values))

In [ ]:
# train set absolute values
train_num_skeweness_values = [abs(i) for i in train_num_skeweness_values]
#train_num_skeweness_values

# test set absolute values
test_num_skeweness_values = [abs(i) for i in train_num_skeweness_values]
#test_num_skeweness_values

In [ ]:
train_skewness_df = pd.DataFrame({'columns':train_num_cols_with_na, 'skew_value':train_num_skeweness_values})
train_skewness_df.head(2)

In [ ]:
test_skewness_df = pd.DataFrame({'columns':test_num_cols_with_na, 'skew_value':test_num_skeweness_values})
test_skewness_df.head(2)

In [ ]:
print(len(test_num_cols_with_na))
print(len(test_num_skeweness_values))

In [ ]:
# create skewness bins
# symmetric = < 0.5
# slighty_skewed  = 0.5 - 1
# highly_skewed = > 1
train_skeweness_labels = ['symmetric','slighty_skewed', 'highly_skewed']
train_skeweness_bins = [0, 0.5, 1, float('inf')]
train_skewness_df['skew_bin'] = pd.cut(train_skewness_df['skew_value'], bins=train_skeweness_bins, labels=train_skeweness_labels)

# test set
test_skeweness_labels = ['symmetric','slighty_skewed', 'highly_skewed']
test_skeweness_bins = [0, 0.5, 1, float('inf')]
test_skewness_df['skew_bin'] = pd.cut(test_skewness_df['skew_value'], bins=test_skeweness_bins, labels=test_skeweness_labels)

In [ ]:
train_skewness_df.head(2)

In [ ]:
test_skewness_df.head(2)

In [ ]:
# symmetric numerical columns with missing values.
# Missing values will be replaced with mean.
# train set
train_symmetric_num_cols = train_skewness_df.loc[train_skewness_df['skew_bin'] == 'symmetric']
train_symmetric_num_cols = train_symmetric_num_cols['columns'].to_list()
print(len(train_symmetric_num_cols))

# test set
test_symmetric_num_cols = test_skewness_df.loc[test_skewness_df['skew_bin'] == 'symmetric']
test_symmetric_num_cols = test_symmetric_num_cols['columns'].to_list()
print(len(test_symmetric_num_cols))

In [ ]:
# asymmetric numerical columns with missing values.
# Missing values will be replaced with median.
# train set
train_asymmetric_num_cols = train_skewness_df.loc[train_skewness_df['skew_bin'] != 'symmetric']
train_asymmetric_num_cols = train_asymmetric_num_cols['columns'].to_list()
print(len(train_asymmetric_num_cols))

# test set
test_asymmetric_num_cols = test_skewness_df.loc[test_skewness_df['skew_bin'] != 'symmetric']
test_asymmetric_num_cols = test_asymmetric_num_cols['columns'].to_list()
print(len(test_asymmetric_num_cols))

#### Imputing missing values

for i in symmetric_num_cols:
    mean = train_df_unqique[i].mean()
    print(mean)

for i in asymmetric_num_cols:
    median = train_df_unqique[i].median()
    print(median)

In [ ]:
print(train_df.isna().sum().sum())
print(test_df.isna().sum().sum())

In [ ]:
# function takes dataframe and imputation method as input parameters
def missing_value_imputation(df1, df2):
    # mean imputation
    for i in train_symmetric_num_cols:
        mean = df1[i].mean()
        df1[i].fillna(mean, inplace=True)
    for i in test_symmetric_num_cols:
        mean = df2[i].mean()
        df2[i].fillna(mean, inplace=True)

    # median imputation
    for i in train_asymmetric_num_cols:
        median = df1[i].median()
        df1[i].fillna(median, inplace=True)
    for i in test_asymmetric_num_cols:
        median = df2[i].median()
        df2[i].fillna(median, inplace=True)

In [ ]:
missing_value_imputation(train_df, test_df)

In [ ]:
print(train_df.isna().sum().sum())
print(test_df.isna().sum().sum())

### Debt Analysis

- Explore the top 2,500 locations where the percentage of households with a second mortgage is the highest and percent ownership is above 10 percent. Visualize using geo-map. You may keep the upper limit for the percent of households with a second mortgage to 50 percent
- Use the following bad debt equation:
    - Bad Debt = P (Second Mortgage ∩ Home Equity Loan)
    - Bad Debt = second_mortgage + home_equity - home_equity_second_mortgage

In [ ]:
from pandasql import sqldf
q1 = 'select state,place,pct_own,second_mortgage,lat,lng from train_df where pct_own >0.10 and second_mortgage <0.5 order by second_mortgage DESC LIMIT 2500;'
pysqldf = lambda q: sqldf(q, globals())
train_df_location_mort_pct = pysqldf(q1)

In [ ]:
train_df_location_mort_pct.head(2)

In [ ]:
pip install vega_datasets

In [ ]:
# import state boundaries
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, feature='states')

background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=500,
    height=300
)

points = alt.Chart(train_df_location_mort_pct).mark_circle().encode(
    longitude='lng:Q',
    latitude='lat:Q',
    size=alt.value(10),
    tooltip=['place', 'state', 'pct_own'],
    color='state'
).project(
    "albersUsa"
).properties(
    width=500,
    height=400
)

background + points

##### Use the following bad debt equation: Bad Debt = P (Second Mortgage ∩ Home Equity Loan) Bad Debt = second_mortgage + home_equity - home_equity_second_mortgage c) Create pie charts to show overall debt and bad debt

In [ ]:
train_df.head(2)

In [ ]:
train_df['bad_debt'] = train_df['second_mortgage'] + train_df['home_equity'] - train_df['home_equity_second_mortgage']

# test set
test_df['bad_debt'] = test_df['second_mortgage'] + test_df['home_equity'] - test_df['home_equity_second_mortgage']

In [ ]:
train_df.head(2)

In [ ]:
train_df['bad_debt_bins'] = pd.cut(train_df['bad_debt'], bins=[0, 0.1, 1], labels=['less than 50%', 'greater than or equal to 50%'])
train_df.groupby(['bad_debt_bins']).size().plot(kind='pie', subplots=True, startangle=90, autopct='%1.1f%%')
plt.axis('equal')

plt.show()

# test set
test_df['bad_debt_bins'] = pd.cut(test_df['bad_debt'], bins=[0, 0.1, 1], labels=['less than 50%', 'greater than or equal to 50%'])

##### Create box and whisker plot and analyze the distribution for 2nd mortgage, home equity, good debt, and bad debt for different cities

In [ ]:
# Georgia city comparison dataframe
duluth_df = train_df.loc[(train_df['city'] == 'Duluth') & (train_df['state'] == 'Georgia')]
lilburn_df = train_df.loc[(train_df['city'] == 'Lilburn') & (train_df['state'] == 'Georgia')]
ga_duluth_lilburn_df = pd.concat([duluth_df, lilburn_df])
ga_duluth_lilburn_df.head(2)

In [ ]:
base = alt.Chart(ga_duluth_lilburn_df).mark_boxplot().encode(
    x='city',
    y='second_mortgage'
).properties(
    width=200,
    height=400
)

base | base.encode(y='home_equity') | base.encode(y='debt') | base.encode(y='bad_debt')

##### Create a collated income distribution chart for family income, house hold income, and remaining income

In [ ]:
# Family income distribution
sns.displot(train_df['family_mean'])
plt.title('Family income distribution')
plt.show()

In [ ]:
# Household income distribution
sns.displot(train_df['hi_mean'])
plt.title('Household income distribution')
plt.show()

In [ ]:
# Remaining income
sns.displot((train_df['family_mean']) - (train_df['hi_mean']))
plt.title('Remaining income distribution')
plt.show()

##### Perform EDA and come out with insights into population density and age. You may have to derive new fields (make sure to weight averages for accurate measurements): 

In [ ]:
# Population density
base = alt.Chart(train_df).mark_circle().encode(
    x='pop',
    y='count()'
).properties(
    width=400,
    height=400
)

base | base.encode(x='female_pop') | base.encode(x='male_pop')

In [ ]:
# Age density
base = alt.Chart(train_df).mark_circle().encode(
    x='female_age_mean',
    y='female_age_samples'
).properties(
    width=400,
    height=400
)

base | base.encode(x='male_age_mean', y='male_age_samples')

##### Use pop and ALand variables to create a new field called population density. Use male_age_median, female_age_median, male_pop, and female_pop to create a new field called median age.

In [ ]:
# derive population density https://www.nationalgeographic.org/encyclopedia/population-density/
train_df['pop_density'] = train_df['pop'] / train_df['ALand']

# median age
train_df['median_age'] = (train_df['male_age_median'] + train_df['female_age_median']) / 2

# test set
test_df['pop_density'] = test_df['pop'] / test_df['ALand']
# median age
test_df['median_age'] = (test_df['male_age_median'] + test_df['female_age_median']) / 2

In [ ]:
train_df.head()

In [ ]:
# Population density
base = alt.Chart(train_df).mark_line().encode(
    x='pop',
    y='pop_density'
).properties(
    width=400,
    height=400
)

base

In [ ]:
# Combined age density
base = alt.Chart(train_df).mark_line().encode(
    x='median_age',
    y='pop_density'
).properties(
    width=400,
    height=400
)

base

##### Create bins for population into a new variable by selecting appropriate class interval so that the number of categories don’t exceed 5 for the ease of analysis.

In [ ]:
labels = ['low', 'medium', 'high']

# train set
train_df['pop_bin'] = pd.cut(train_df['pop'], bins=3, labels=labels)

# test set
test_df['pop_bin'] = pd.cut(test_df['pop'], bins=3, labels=labels)
train_df.shape, test_df.head()

In [ ]:
train_df[['married', 'separated', 'divorced', 'pop_bin']]

In [ ]:
# married, separated, divorces vs pop_bin density
base = alt.Chart(train_df).mark_bar().encode(
    x='pop_bin',
    y='married'
).properties(
    width=400,
    height=400
)

base | base.encode(y='separated') | base.encode(y='divorced')

##### Please detail your observations for rent as a percentage of income at an overall level, and for different states.
Perform correlation analysis for all the relevant variables by creating a heatmap. Describe your findings.

In [ ]:
# state family mean
state_family_income_avg = train_df.groupby(by='state')['family_mean'].agg(['mean'])
state_family_income_avg.head()

In [ ]:
# state rent mean
state_rent_avg = train_df.groupby(by='state')['rent_mean'].agg(['mean'])
state_rent_avg.head()

In [ ]:
rent_income_perc = state_rent_avg['mean']/state_family_income_avg['mean']
rent_income_perc.head()

In [ ]:
# national rent percentage of family income
sum(train_df['rent_mean']) / sum(train_df['family_mean'])

##### The economic multivariate data has a significant number of measured variables. The goal is to find where the measured variables depend on a number of smaller unobserved common factors or latent variables. 
- Each variable is assumed to be dependent upon a linear combination of the common factors, and the coefficients are known as loadings. Each measured variable also includes a component due to independent random variability, known as “specific variance” because it is specific to one variable. Obtain the common factors and then plot the loadings. Use factor analysis to find latent variables in our dataset and gain insight into the linear relationships in the data.

Following are the list of latent variables:
- Highschool graduation rates
- Median population age
- Second mortgage statistics
- Percent own
- Bad debt expense

###### Factor Analysis
- https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FactorAnalysis.html
- https://factor-analyzer.readthedocs.io/en/latest/

In [ ]:
from sklearn.decomposition import FactorAnalysis
from factor_analyzer import FactorAnalyzer

In [ ]:
#pip install factor_analyzer

In [ ]:
f_analyzer = FactorAnalyzer(n_factors=5)
f_analyzer.fit_transform(train_df.select_dtypes(exclude=('object', 'category')))
f_analyzer.loadings_[0]

In [ ]:
train_df.columns

In [ ]:
relevant_columns = ['ALand', 'hi_mean', 'family_mean', 'second_mortgage', 'home_equity', 
                   'married', 'separated', 'divorced', 'pop', 'bad_debt', 'median_age', 'hc_mortgage_mean']
len(relevant_columns)

In [ ]:
# corr matrix
relevant_cols_corr = train_df[relevant_columns].corr()

In [ ]:
plt.figure(figsize=(10, 10))
sns.heatmap(relevant_cols_corr, annot=True, cmap='RdYlBu_r')
plt.show()

## 4. Modeling

### Build a linear Regression model to predict the total monthly expenditure for home mortgages loan. Column **hc_mortgage_mean** is predicted variable. This is the mean monthly mortgage and owner costs of specified geographical location. *Note*: Exclude loans from prediction model which have NaN (Not a Number) values for hc_mortgage_mean.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score

In [ ]:
train_df['hc_mortgage_mean'].isna().sum().sum()

In [ ]:
test_df['hc_mortgage_mean'].isna().sum().sum()

In [ ]:
%matplotlib inline

import matplotlib as mpl

#import seaborn as sns

#import matplotlib.pyplot as plt

import statsmodels.formula.api as smf

import statsmodels.graphics.api as smg

#import pandas as pd

#import numpy as np

import patsy

from statsmodels.graphics.correlation import plot_corr

#from sklearn.model_selection import train_test_split

plt.style.use('seaborn')

In [ ]:
# multiple linear regression using Statmodels Formula Api
multiLinearModel = smf.ols(formula= 'hc_mortgage_mean ~ ALand + hi_mean + family_mean + second_mortgage + home_equity + married + separated + divorced + pop + bad_debt',data=train_df)
multiLinearModResult = multiLinearModel.fit()

In [ ]:
print(multiLinearModResult.summary())

In [ ]:
# prepare data for modeling
train_cols_to_drop = []
for i in train_df.columns:
    if ((train_df[i].dtype != 'float64') & (train_df[i].dtype != 'int64')):
        train_cols_to_drop.append(i)
train_cols_to_drop

In [ ]:
# prepare data for modeling
test_cols_to_drop = []
for i in test_df.columns:
    if ((test_df[i].dtype != 'float64') & (test_df[i].dtype != 'int64')):
        test_cols_to_drop.append(i)
test_cols_to_drop

In [ ]:
# define data sets for model

x_train = train_df.drop(['hc_mortgage_mean', 'COUNTYID','STATEID','state', 'state_ab', 'city', 'place', 'type', 'zip_code', 'area_code', 'bad_debt_bins', 'pop_bin'], axis=1)
y_train = train_df['hc_mortgage_mean']

# test set
x_test = test_df.drop(['hc_mortgage_mean', 'COUNTYID','STATEID','state', 'state_ab', 'city', 'place', 'type', 'zip_code', 'area_code', 'bad_debt_bins', 'pop_bin'], axis=1)
y_test = test_df['hc_mortgage_mean']

from sklearn import preprocessing
minmaxScaler = preprocessing.MinMaxScaler()
x_train_tran = pd.DataFrame(minmaxScaler.fit_transform(x_train))
x_test_tran = pd.DataFrame(minmaxScaler.fit_transform(x_test))

In [ ]:
# benchmark model
from sklearn.linear_model import LinearRegression

# fit a linear regression model to the data
bench_mark_lr_model = LinearRegression()
bench_mark_lr_model.fit(x_train_tran, y_train)
bench_mark_lr_model.coef_

In [ ]:
bench_mark_lr_model.summary()

##### Dimensionality reduction

In [ ]:
# dimensionality reduction using pca
from sklearn.decomposition import PCA
import time
t0 = time.time()
pca = PCA().fit(x_train_tran)
t1 = time.time()
print('PCA fitting time:', round(t1-t0, 3), 's')